<a href="https://colab.research.google.com/github/CodewithTanzeel/BlackByte-ML/blob/main/fraud_svm_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# Imports
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline as SKPipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

RND = 42


In [ ]:
# =========================
# Data Split
# =========================
df = pd.read_csv("/content/fraud_data_preparation.csv")
TARGET = "Class"

X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RND
)


In [ ]:
# =========================
# Define Pipelines
# =========================
svm = SVC(kernel='rbf', random_state=RND)

# Pre-SMOTE pipeline
pre_smote_pipeline = SKPipeline([
    ('scaler', StandardScaler()),
    ('svc', svm)
])

# Post-SMOTE pipeline (SMOTE inside CV)
post_smote_pipeline_cv = ImbPipeline([
    ('smote', SMOTE(random_state=RND)),
    ('scaler', StandardScaler()),
    ('svc', svm)
])


NameError: name 'SVC' is not defined

In [ ]:
# =========================
# 5-Fold Cross Validation
# =========================
scoring = {'accuracy':'accuracy','precision':'precision','recall':'recall','f1':'f1'}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RND)

cv_results_pre = cross_validate(pre_smote_pipeline, X_train, y_train, cv=skf, scoring=scoring)
cv_results_post = cross_validate(post_smote_pipeline_cv, X_train, y_train, cv=skf, scoring=scoring)

def summarize_cv(results, label):
    print(f"\n--- {label} ---")
    for metric in scoring.keys():
        vals = results[f'test_{metric}']
        print(f"{metric}: {vals.mean():.4f} ± {vals.std():.4f}")

summarize_cv(cv_results_pre, "Pre-SMOTE CV")
summarize_cv(cv_results_post, "Post-SMOTE CV")


NameError: name 'StratifiedKFold' is not defined

In [ ]:
# =========================
# Final Training & Evaluation
# =========================
# Pre-SMOTE
pre_smote_pipeline.fit(X_train, y_train)
y_pred_pre = pre_smote_pipeline.predict(X_test)

# Post-SMOTE
sm = SMOTE(random_state=RND)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

scaler = StandardScaler().fit(X_train_res)
X_train_res_scaled = scaler.transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

svm_post = SVC(kernel='rbf', random_state=RND)
svm_post.fit(X_train_res_scaled, y_train_res)
y_pred_post = svm_post.predict(X_test_scaled)

def evaluate(y_true, y_pred, label):
    print(f"\n--- {label} ---")
    print(classification_report(y_true, y_pred, digits=4))
    cm = confusion_matrix(y_true, y_pred)
    ConfusionMatrixDisplay(cm).plot(cmap="Blues")
    plt.title(f"Confusion Matrix - {label}")
    plt.show()

evaluate(y_test, y_pred_pre, "Pre-SMOTE")
evaluate(y_test, y_pred_post, "Post-SMOTE")


NameError: name 'pre_smote_pipeline' is not defined

In [ ]:
# =========================
# Fraud Alert Simulation
# =========================
def fraud_alert(pred):
    return "RAISE ALERT" if pred == 1 else "APPROVE"

sample_preds = y_pred_post[:10]
print([fraud_alert(p) for p in sample_preds])


NameError: name 'y_pred_post' is not defined